1. Импортируйте библиотеку pandas . Загрузите датасет bookings.csv с разделителем ;. Проверьте размер таблицы, типы переменных, а затем выведите первые 7 строк, чтобы посмотреть на данные. 


In [2]:
import pandas as pd 
import numpy as np

In [3]:
df = pd.read_csv("bookings.csv", sep=';', parse_dates=['Reservation status_date'])
df.head(7)

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


2. Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.


In [4]:
df.columns = [col_name.lower().replace(' ', '_') for col_name in df.columns]
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [5]:
df.insert(0, 'id', range(1, len(df) + 1))


3. Пользователи из каких стран совершили наибольшее число успешных бронирований? Укажите топ-5.

In [6]:
successful_orders = df.query('is_canceled == 0').groupby('country')['id'].count().sort_values(ascending=False)
successful_orders.head(5)

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: id, dtype: int64

4. На сколько ночей в среднем бронируют отели разных типов?

In [7]:
res = df.groupby('hotel')['stays_total_nights'].mean()
res

hotel
City Hotel      2.978142
Resort Hotel    4.318547
Name: stays_total_nights, dtype: float64

5. Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга. Сколько подобных наблюдений встретилось в датасете?

In [8]:
res = df.query('assigned_room_type != reserved_room_type')['id'].count()
res

np.int64(14917)

6.1. На какой месяц чаще всего успешно оформляли бронь в 2016? Изменился ли самый популярный месяц в 2017?

In [9]:
import datetime
df['reservation_status_date_month'] = pd.to_datetime(df['reservation_status_date']).dt.month
df['reservation_status_date_year'] = pd.to_datetime(df['reservation_status_date']).dt.year

year = 2016
d_2016 = df.query(f'reservation_status_date_year == {year} and is_canceled == False').groupby('reservation_status_date_month')['id'].count().sort_values(ascending=False).to_dict()
year += 1
d_2017 = df.query(f'reservation_status_date_year == {year} and is_canceled == False').groupby('reservation_status_date_month')['id'].count().sort_values(ascending=False).to_dict()

successful_month_2016 = datetime.date(1900, max(d_2016, key=lambda k: d_2016[k]), 1).strftime('%B')
successful_month_2017 = datetime.date(1900, max(d_2017, key=lambda k: d_2017[k]), 1).strftime('%B')
successful_month_2016, successful_month_2017

('October', 'May')

6.2. Сгруппируйте данные по годам и проверьте, на какой месяц бронирования отеля типа City Hotel отменялись чаще всего в каждый из периодов.

In [16]:
q = df.query('is_canceled == True').groupby('reservation_status_date_year')['id'].count().idxmax()
q


np.int32(2016)

7. Посмотрите на числовые характеристики трёх переменных: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [11]:
d = {k: df[k].mean() for k in ['adults', 'children', 'babies']}
mx = max(d, key=lambda k: d[k])
mx

'adults'

8. Создайте колонку total_kids, объединив children и babies. Отели какого типа в среднем пользуются большей популярностью у клиентов с детьми?

9. Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids),и False – в противном случае.

In [12]:
df['total_kids'] = df['children'] + df['babies']
df['has_kids'] = df['total_kids'] > 0

popularity = df.query('has_kids == True').groupby('hotel')['id'].count().sort_values(ascending=False).to_dict()
mx = max(popularity, key=lambda k: popularity[k])
mx

'City Hotel'

10. Посчитайте отношение количества ушедших пользователей к общему количеству клиентов, выраженное в процентах (churn rate). Укажите, среди какой группы показатель выше.

In [25]:
df['rest_days'] = pd.to_datetime(df['reservation_status_date']) - pd.to_datetime(df['arrival_full_date'])
df['pass_user'] = df['rest_days'].dt.days < df['stays_total_nights']
print(df.query('pass_user == True').shape[0] / df.shape[0] * 100)
df['total_kids'] = df['children'] + df['babies']
df['has_kids'] = df['total_kids'] > 0
print(df.query('pass_user == True and has_kids == True').shape[0], ' ', df.query('pass_user == True and has_kids == False').shape[0] )

37.03492754837088
3263   40953
